In [1]:
#=========================================================================================================
#================================ 0. MODULE

# Classic
import numpy as np
import pandas as pd

# Pytorch framework
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.nn.init as init
import torchvision.models as models
from torch.autograd import Function
from torch.autograd import Variable
import torchvision.models as models

# Utils
from typing import List, Tuple
from itertools import product as product
from math import sqrt
from math import pi

# Perso
import sys
sys.path.append('/home/hugoperrin/Bureau/X/Cours/Data_camp/mars_craters/perso/')
from object_detector import *

device = 'cuda:0'

BATCH_SIZE = 16

net = create_SSD(config=config, device=device)
x = torch.rand(BATCH_SIZE, 3, 224, 224).to(device)
loc, conf, prior = net(x)

print('location shape: {}'.format(loc.shape))
print('Confidence shape: {}'.format(conf.shape))
print('Prior shape: {}'.format(prior.shape))

location shape: torch.Size([16, 2100, 3])
Confidence shape: torch.Size([16, 2100, 1])
Prior shape: torch.Size([2100, 3])


In [2]:
image_idx = 10

# Computed predicted circles
predicted_circles = torch.clamp(prior.add(loc[image_idx]), min=0)
predicted_conf = conf[image_idx]

# Get ground truth
path = '/home/hugoperrin/Bureau/X/Cours/Data_camp/mars_craters/data/labels_train_mini.csv'
labels = pd.read_csv(path)
truth = labels[labels['i'] == image_idx].drop(['i'], axis=1).values
true_circles = torch.tensor(truth, dtype=torch.float).to(device) / 224

In [3]:
predicted_circles.shape, true_circles.shape, predicted_conf.shape

(torch.Size([2100, 3]), torch.Size([4, 3]), torch.Size([2100, 1]))

In [4]:
# # Testing IoU
# circles_A = np.array([[68, 212, 5],
#                       [10, 10, 20],
#                       [10, 10, 20],
#                       [10, 10, 0],
#                       [39, 20, 20],
#                       [0, 0, 0],
#                       [45, 189, 12.5],
#                       [11, 11, 5],
#                       [60, 32, 5]])
# circles_A = torch.tensor(circles_A, dtype=torch.float) / 224

# conf = np.array([1, 1, 0.7, 1, 0.3, 0.2, 0.6, 0.7, 0.15])
# conf = torch.tensor(conf, dtype=torch.float)

# circles_B = np.array([[40, 10, 9],
#                       [10, 10, 19],
#                       [170, 10, 8],
#                       [60, 32, 5]])
# circles_B = torch.tensor(circles_B, dtype=torch.float) / 224

In [5]:
matches = match(predicted_circles, true_circles, threshold=0.4)
image_loss = multi_circle_loss(predicted_circles, true_circles, predicted_conf, matches, 2)

In [ ]:
image_loss